In [1]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress
from scipy.signal import butter,filtfilt, find_peaks

In [2]:
disp = 0.1

In [3]:
df_exp_PB = pd.read_csv('PB18_UCS_2622.csv')

# Filter the data, and plot both the original and filtered signals.
stress = df_exp_PB['Stress [MPa]']             # Stress
strain = df_exp_PB['Strain Ax [-]']            # Strain


# Plot maximum strength
y = stress.max()                                                                               # Find maximum stress
max_index = stress.argmax()
# print(max_index)                                                                             # 2357 is the index of the maximum value
x = strain[max_index]                                                                          # Apply the maximum index of the stress to the strain


index_min = stress[stress > 0.3].index[0]
df_exp_PB_short = df_exp_PB.iloc[index_min : max_index + 320]

stress = df_exp_PB_short['Stress [MPa]']

strain = df_exp_PB_short['Strain Ax [-]']



stress1 = stress.to_numpy()

strain1 = strain.to_numpy()

In [4]:
## Fracture strength: point of strain where the material physically separates. 
# At this point, the strain reaches its maximum value and the material actually fractures, even though the corresponding stress may be less than the ultimate strength at this point
fract_strain =  strain1.max()
max_index_fract = strain1.argmax()
fract_stress = stress1[max_index_fract]
# fract_stress = np.interp(fract_strain, strain1, stress1)
FractS = f'The fracture strength is {round(fract_stress,1)} MPa'
print(fract_strain, fract_stress)

0.00545158 22.918435


In [5]:
###ONLY FOR LINEAR REGRESSION
# Plot the young modulus slope
stress_range = stress1[(stress1 >= 12) & (stress1 <= 17)]                               # stress filtered for values more than 15 less 33
strain_range = strain1[(stress1 >= 12) & (stress1 <= 17)]                               # strain filtered for values more than 15 less 33

## Linear regression to fit the linear slope corresponding to the Young's
linear_regression_output = linregress(strain_range, stress_range)
print(linear_regression_output)
E = linear_regression_output[0]
b = linear_regression_output[1]
print(f'The elastic modulus is {round(E,1)} MPa')

# stress_offset = E*strain+b
stress_offset = E*(strain1) + b - disp                                                                 # y=mx+b


ElasticM = f'The elastic modulus is {round(E/1000,1)} GPa'


## Find yield strength
for i in range(2, len(strain1)):
    if stress_offset[i] >= stress1[i]:
        sx1 = strain1[i-1]
        sy1 = stress1[i-1]
        sy2 = stress1[i]
        sx2 = strain1[i]

        ox1 = strain1[i-1]
        oy1 = stress_offset[i-1]
        ox2 = stress1[i]
        oy2 = stress_offset[i]
        break

# print(sx1, sy1, sx2, sy2, ox1, oy1, ox2, oy2)

x1 = ox1
y1 = oy1
x2 = ox2
y2 = oy2

x3 = sx1
y3 = sy1
x4 = sx2
y4 = sy2

YS = ( (x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4) ) / ( ( (x1-x2)*(y3-y4))- ( (y1-y2)*(x3-x4)))
print(YS)

xi = (YS-b) / (E)
yi = YS

print('(xi,yi)',xi,yi)

LinregressResult(slope=9616.198485876615, intercept=-18.450885990221327, rvalue=0.9999646537271059, pvalue=0.0, stderr=4.723543649025212, intercept_stderr=0.01619287148848673)
The elastic modulus is 9616.2 MPa
20.473378366194403
(xi,yi) 0.0040477808786480544 20.473378366194403


In [6]:
### PLOTTING
## Plot the data
ax = px.line(df_exp_PB, x = 'Strain Ax [-]', y = 'Stress [MPa]', width=700, height=700, title='PB16_UCS - Full test')

ax.show() 

# ax.write_html("Graphs/PB18_UCS/PB18_FullTest.html")

In [7]:
### All info
# plt.plot(strain1,stress_offset)
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))
fig.add_trace(go.Scatter(x=strain1, y=stress_offset))

fig.add_annotation(x=x, y=y,
            text="Strain={:.4f}, Stress={:.4f}".format(x, y),
            showarrow=True,
            xanchor="right",
            arrowhead=1)
            
fig.add_annotation(x=xi, y=2,
            text=ElasticM,
            showarrow=False,
            arrowhead=1)

fig.add_annotation(x=xi, y=0,
            text=FractS,
            showarrow=False,
            arrowhead=1)


fig.add_annotation(x=xi, y=yi,
            text="Strain={:.4f}, Stress={:.4f}".format(x1, y1),
            xanchor="right",
            showarrow=True,
            arrowhead=1)

fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
    title="All info",
    xaxis_title="Strain Ax [-]",
    yaxis_title="Stress [MPa]",
    font=dict(
        family="New Times Roman",
        size=15,
        color="RebeccaPurple"))


fig.update_traces(textposition='top center')
fig.show()

# fig.write_html("Graphs/PB18_UCS/PB18_Results.html")

In [8]:
### All info
# plt.plot(strain1,stress_offset)
fig = go.Figure()
fig.add_trace(go.Scatter(x=strain1, y=stress1))
# fig.add_trace(go.Scatter(x=strain1, y=stress_offset))

# fig.add_annotation(x=x, y=y,
#             text="Strain={:.4f}, Stress={:.4f}".format(x, y),
#             showarrow=True,
#             xanchor="right",
#             arrowhead=1)
            
# fig.add_annotation(x=xi, y=2,
#             text=ElasticM,
#             showarrow=False,
#             arrowhead=1)

# fig.add_annotation(x=xi, y=0,
#             text=FractS,
#             showarrow=False,
#             arrowhead=1)


# fig.add_annotation(x=xi, y=yi,
#             text="Strain={:.4f}, Stress={:.4f}".format(x1, y1),
#             xanchor="right",
#             showarrow=True,
#             arrowhead=1)

# fig.update_layout(showlegend=False, autosize=False, width=700, height=700, 
#     title="All info",
#     xaxis_title="Strain Ax [-]",
#     yaxis_title="Stress [MPa]",
#     font=dict(
#         family="New Times Roman",
#         size=15,
#         color="RebeccaPurple"))

fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)
fig.update_layout(template="simple_white", width=700, height=700, yaxis=dict(range=[0, 270]),# customize font and legend orientation & position
                  xaxis=dict(range=[0, 0.025]), title='PB18 - UCS',
    font=dict(
        family="Times New Roman",
        size=20,
        color="Black"),
    legend=dict(title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"))
# fig.update_traces(textposition='top center')
fig.show()

# fig.write_html("Graphs/PB2_UCS/PB2_Results.html")